In [1]:
import pandas as pd
import json
import requests
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def create_embedding(text_list):
    r = requests.post("http://localhost:11434/api/embed",json={
        "model":"bge-m3",
        "input":text_list
    })

    embedding = r.json()['embeddings']
    return embedding

In [2]:
data = []

with open('data.json','r') as f:
    data = json.load(f)


In [3]:
len(data)

1398

In [4]:
# df = pd.DataFrame.from_records(data) for first time 
df = joblib.load('embeddings.joblib')

NameError: name 'joblib' is not defined

In [ ]:
incoming_query = "who is markaroni? what does he do? "

question_embedding = create_embedding([incoming_query])[0]

similarities = cosine_similarity(np.vstack(df['embedding']),[question_embedding]).flatten()

max_idx = similarities.argsort()[::-1][0:10]

new_df = df.loc[max_idx]

print(new_df[['title','number','text']])


                            title  number         text
458  How good was marco vanbasten       1    he scores
702  How good was marco vanbasten       1      himself
629  How good was marco vanbasten       1   he will be
714  How good was marco vanbasten       1           he
717  How good was marco vanbasten       1           he
722  How good was marco vanbasten       1           he
688  How good was marco vanbasten       1           he
749  How good was marco vanbasten       1     that guy
656  How good was marco vanbasten       1       he was
635  How good was marco vanbasten       1       he was


In [ ]:
#the data is currently stored in data.json with overriding the data without the embeddings
#we can also save the process that is store the data frame using joblib

import joblib

In [ ]:
# joblib.dump(df,'embeddings.joblib')

In [ ]:
incoming_query = "who is lionel messi"

question_embedding = create_embedding([incoming_query])[0]

similarities = cosine_similarity(np.vstack(df['embedding']),[question_embedding]).flatten()

max_idx = similarities.argsort()[::-1][0:50]

new_df = df.loc[max_idx]

# print(new_df[['title','number','text']])


prompt = f'''You are a knowledgeable football expert assistant helping users learn about football history and players based on Markaroni's video content.

## Context Data
Below are the most relevant video chunks from Markaroni's football videos:

{new_df[['title','number','text','start','end']].to_json()}

## User Question
{incoming_query}

## Instructions
1. Analyze the provided video chunks carefully
2. Answer the user's question directly and conversationally
3. Use specific information from the chunks to support your answer
4. If the question is unrelated to the provided content, politely inform the user that you don't have relevant information
5. Include video references in this format: "Video: [title] at [MM:SS]"
6. Convert timestamps from seconds to MM:SS format (e.g., 90 seconds = 1:30)
7. Structure your response clearly with paragraphs for readability
8. Be concise but informative - aim for 3-5 sentences unless more detail is needed

## Response Format
Provide your answer in a natural, conversational tone as if speaking directly to the user. Include relevant timestamps and video titles 
to help them find the original content.Don't try to answer the questions just tell
the user in which video and at what time he will get the relevent answers 
simply give output as video_name - Time_stamp => the info you can get that is it do not ask
any counter question or suggestion
'''

with open('prompt.txt','w') as f:
    f.write(prompt)


In [ ]:
import requests
def inference(prompt):
    r = requests.post('https://localhost:11434/api/embed',json={
            "model":"deepseek-r1",
            'prompt':prompt,
            "stream":False
            })
    
    response = r.json()
    print(response)



_IncompleteInputError: incomplete input (494691780.py, line 2)